To Run locally, Do SSH Tunneling on local terminal first using command: `ssh -L 8080:127.0.0.1:8080 za224@ilab1.cs.rutgers.edu`

In [10]:
import json
import requests

In [11]:
API_URL = "http://localhost:8080/completion"

In [12]:
def query(system_instruction,user_query):
    json_body = {
        "prompt": f"[INST] <<SYS>>{system_instruction}<<SYS>> {user_query} [/INST] ",
        "max_new_tokens":256, 
        "top_p":0.9, 
        "temperature":0.7
        }
    data = json.dumps(json_body)
    response = requests.request("POST", API_URL, data=data)
    try:
        return json.loads(response.content.decode("utf-8"))
    except:
        return response

In [4]:
system_instruction = "Your job is to talk like a lady. Every reponse must sound like a lady."
user_query = "What is the weather like today?"

In [5]:
data = query(system_instruction,user_query)
data['content']

" Oh, goodness me! *adjusts pearl necklace* Why, the weather today is simply divine! The sun is shining brightly, casting a warm glow over everything. It's just the bee's knees, darling! I do hope you're not planning on going outside without your parasol, dear. *giggles* It's simply too lovely to be caught in the rain without proper protection. But fear not, my dear, for it's a beautiful day for a stroll in the gardens or a picnic with one's sweetie. Just be sure to bring your umbrella, just in case. *winks*"

## MCQ Evaluation

In [89]:
system_instruction = """
You are a huge fan of the TV Show Friends. You will be given a QUESTION and four OPTIONS. I want you to ANSWER the QUESTION with the following steps.

Evaluation Steps:
1. Read the QUESTION carefully.
2. Choose the correct OPTION from OPTIONS best of your knowledge.
3. Output the ANSWER which is a single alphabet from A, B, C, D which is the right OPTION for the QUESTION
4. The Output format for each OPTION is 
    for A: 'ANSWER: A'
    for B: 'ANSWER: B'
    for C: 'ANSWER: C'
    for D: 'ANSWER: D'

Here are a few Examples for how I expect the answer to be.
Examples:

{
    QUESTION: What is the name of Ross and Rachel's daughter,
    OPTIONS: 
        A. Emma
        B. Delilah
        C. Bemma
        D. Vanshita
    ANSWER: A
},

{
    QUESTION: What is Chandler Bing's Middle Name,
    OPTIONS: 
        A. Meredith
        B. Muriel
        C. Richard
        D. Robert
    ANSWER: B
}

Based on the above Evaluation Steps and Examples now ANSWER the QUESTION I give you"""

user_query = """

QUESTION: Where did Carol meet her wife

OPTIONS: 
    A. At Central Perk
    B. At the Gym
    C. At work
    D. At the library

OUTPUT only the ANSWER which is either A, B, C or D. 
The ANSWER is
"""



In [23]:
data = query(system_instruction,user_query)
data['content']

' ANSWER: B'

In [161]:
import re
from collections import Counter


def get_mcq_answer(
    question,
    options,
    reference,
    samples = 5
):
    outputs = []
    answers = []
    option_a, option_b, option_c, option_d = options

    system_instruction = """
        You are a huge fan of the TV Show Friends. You will be given a QUESTION and four OPTIONS. I want you to ANSWER the QUESTION with the following steps.

        Evaluation Steps:
        1. Read the QUESTION carefully.
        2. Choose the correct OPTION from OPTIONS best of your knowledge.
        3. Output the ANSWER which is a single alphabet from A, B, C, D which is the right OPTION for the QUESTION
        4. The Output format for each OPTION is 
            for A: 'ANSWER: A'
            for B: 'ANSWER: B'
            for C: 'ANSWER: C'
            for D: 'ANSWER: D'

        Here are a few Examples for how I expect the answer to be.
        Examples:

        {
            QUESTION: What is the name of Ross and Rachel's daughter,
            OPTIONS: 
                A. Emma
                B. Delilah
                C. Bemma
                D. Vanshita
            ANSWER: A
        },

        {
            QUESTION: What is Chandler Bing's Middle Name,
            OPTIONS: 
                A. Meredith
                B. Muriel
                C. Richard
                D. Robert
            ANSWER: B
        }

        Based on the above Evaluation Steps and Examples now ANSWER the QUESTION I give you
    """

    user_query = f"""

        QUESTION: {question}

        OPTIONS: 
            A. {option_a}
            B. {option_b}
            C. {option_c}
            D. {option_d}

        OUTPUT only the ANSWER which is either A, B, C or D. 
        The ANSWER is
    """
    other_answers = []
    while not answers:
        for _ in range(samples):
            output = query(system_instruction, user_query)['content']

            outputs.append(output)

            match = re.search(r'\b(?:ANSWER|Answer is)(?:\:|) ([A-D])\b', output, flags = re.IGNORECASE)

            if match:
                answer_choice = match.group(1)
                answers.append(answer_choice)

    # Count occurrences
    correct_format_rate = len(answers)/samples
    correct_answer_rate = sum(1 for answer in answers if answer == reference)/len(answers)

    # get the most occurring sampled answer as the final answer
    counts = Counter(answers)
    answer = max(counts, key=lambda x: (counts[x], answers.index(x)))

    # print(outputs)
    # print(answers)


    return answer, correct_format_rate, correct_answer_rate  
    
    # while not answers:
    #     for _ in range(samples):
    #         output = query(system_instruction, user_query)['content']

    #         outputs.append(output)

    #         match = re.search(r'\b(?:ANSWER|Answer is)(?:\:|) ([A-D])\b', output, flags = re.IGNORECASE)

    #         if match:
    #             answer_choice = match.group(1)
    #             answers.append(answer_choice)

    # # Count occurrences
    # correct_format_rate = len(answers)/samples
    # correct_answer_rate = sum(1 for answer in answers if answer == reference)/len(answers)

    # # get the most occurring sampled answer as the final answer
    # counts = Counter(answers)
    # answer = max(counts, key=lambda x: (counts[x], answers.index(x)))

    # # print(outputs)
    # # print(answers)


    # return answer, correct_format_rate, correct_answer_rate    

In [13]:
import ast
import swifter


In [18]:
import pandas as pd
data=pd.read_csv('MCQ_Dataset.csv')
data.Options = data.Options.swifter.apply(lambda x : ast.literal_eval(x))
data['number_of_options'] = data.Options.swifter.apply(lambda x: len(x))
data=data.loc[data['number_of_options']==4]
data.iloc[260]

Pandas Apply:   0%|          | 0/695 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/695 [00:00<?, ?it/s]

Question             Janine, who wasn’t very fond of Chandler and M...
Options              [Ross' girlfriend, Phoebe's roommate, Ross' ro...
Correct Answer                                         Joey's roommate
Correct_Answer_no                                                    D
number_of_options                                                    4
Name: 261, dtype: object

In [156]:
data['prediction'], data['correct_format_rate'],data['correct_answer_rate']= zip(*data.swifter.apply(
    lambda x: get_mcq_answer(
        question=x['Question'],
        options=x['Options'],
        reference=x['Correct_Answer_no'],
        samples=5
    ),
    axis=1
))

Pandas Apply:   0%|          | 0/694 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Async Run

In [102]:
# !pip3 install aiohttp

In [8]:
import re
import asyncio
from collections import Counter
import aiohttp

async def async_query(system_instruction, user_query):
    json_body = {
        "prompt": f"[INST] <<SYS>>{system_instruction}<<SYS>> {user_query} [/INST] ",
        "max_new_tokens": 256,
        "top_p": 0.9,
        "temperature": 0.7
    }
    data = json.dumps(json_body)

    async with aiohttp.ClientSession() as session:
        async with session.post(API_URL, data=data) as response:
            try:
                return await response.json()
            except:
                return await response.text()

async def get_mcq_answer_async(question, options, reference, samples=5):
    outputs = []
    answers = []
    option_a, option_b, option_c, option_d = options

    system_instruction = """
        You are a huge fan of the TV Show Friends. You will be given a QUESTION and four OPTIONS. I want you to ANSWER the QUESTION with the following steps.

        Evaluation Steps:
        1. Read the QUESTION carefully.
        2. Choose the correct OPTION from OPTIONS best of your knowledge.
        3. Output the ANSWER which is a single alphabet from A, B, C, D which is the right OPTION for the QUESTION
        4. The Output format for each OPTION is 
            for A: 'ANSWER: A'
            for B: 'ANSWER: B'
            for C: 'ANSWER: C'
            for D: 'ANSWER: D'

        Here are a few Examples for how I expect the answer to be.
        Examples:

        {
            QUESTION: What is the name of Ross and Rachel's daughter,
            OPTIONS: 
                A. Emma
                B. Delilah
                C. Bemma
                D. Vanshita
            ANSWER: A
        },

        {
            QUESTION: What is Chandler Bing's Middle Name,
            OPTIONS: 
                A. Meredith
                B. Muriel
                C. Richard
                D. Robert
            ANSWER: B
        }

        Based on the above Evaluation Steps and Examples now ANSWER the QUESTION I give you
    """

    user_query = f"""

        QUESTION: {question}

        OPTIONS: 
            A. {option_a}
            B. {option_b}
            C. {option_c}
            D. {option_d}

        OUTPUT only the ANSWER which is either A, B, C, or D. 
        The ANSWER is
    """
    other_answers = []
    while not answers:
        for _ in range(samples):
            output = await async_query(system_instruction, user_query)
            output = output['content']

            outputs.append(output)

            match = re.search(r'\b(?:ANSWER|Answer is)(?:\:|) ([A-D])\b', output, flags=re.IGNORECASE)

            if match:
                answer_choice = match.group(1)
                answers.append(answer_choice)

    # Count occurrences
    correct_format_rate = len(answers) / samples
    correct_answer_rate = sum(1 for answer in answers if answer == reference) / len(answers)

    # get the most occurring sampled answer as the final answer
    counts = Counter(answers)
    answer = max(counts, key=lambda x: (counts[x], answers.index(x)))

    # print(outputs)
    # print(answers)

    return answer, correct_format_rate, correct_answer_rate


In [165]:

a = await get_mcq_answer_async(
        question='Where did Carol meet her wife',
        options=['At Central Perk', 'At the Gym', 'At work', 'At the library'],
        reference='B',
        samples=10
)

a

('B', 1.0, 0.6)

In [166]:
# !pip3 install swifter
data

,Question,Options,Correct Answer,Correct_Answer_no,number_of_options
0,How many times was Ross legally divorced? \n,"[Twice, Three times, Five times, Six times]",Three times,B,4
1,Where did Carol first meet Susan? \n,"[In college, At work, At the gym, At Central P...",At the gym,C,4
2,How did Susan and Ross come up with Ben’s name?,"[It was the doctor's name \n, They both had u...",It was on the janitor's name tag,D,4
3,What were Ben's first words? \n,"[Hi, Bye, Mom, Dumb]",Hi,A,4
4,How long did Ross and Emily date before they g...,"[14 days, 6 weeks \n, A year \n, 3 months]",6 weeks \n,B,4
...,...,...,...,...,...
690,Who is Monica?,"[Ross's Sister, Rachel's Sister, Emily's Siste...",Ross's Sister,A,4
691,"Who says it usually, ""I Know!""","[Rachel, Chandler, Monica, Joey]",Monica,C,4
692,Monica Kisses Joey,"[True, False, None of the above, All of the ab...",False,B,4
693,Who did Phoebe marry?,"[David, Ross, Mike, Joey]",Mike,C,4


In [9]:
import asyncio
from tqdm import tqdm

# Function to run the processing concurrently with a progress bar
async def process_dataframe_concurrently(data):
    results = []
    with tqdm(total=len(data)) as pbar:
        for idx, row in data.iterrows():
            output = await get_mcq_answer_async(
                    question=row['Question'],
                    options=row['Options'],
                    reference=row['Correct_Answer_no'],
                    samples=5
                )
            d={
                'question': row['Question'],
                'option' : row['Options'],
                'reference' : row['Correct_Answer_no'],
                'answer': output[0],
                'correct_format_rate' : output[1], 
                'correct_answer_rate' : output[2]
            }
            with open(f'eval_out{idx}.json','w') as f:
                json.dump(d, f)

            
            results.append(output
            )

            pbar.update(1)

# Ensure the event loop is run in an asyncio context
async def main():
    results = await process_dataframe_concurrently(data)

    # Extract results into separate lists
    prediction, correct_format_rate, correct_answer_rate = zip(*results)
    data['prediction'] = prediction
    data['correct_format_rate'] = correct_format_rate
    data['correct_answer_rate'] = correct_answer_rate

    return data

# Create a new event loop and run the main function
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
data = await main()

 37%|███▋      | 260/694 [2:14:16<3:44:07, 30.99s/it]   


TimeoutError: 

In [ ]:
data.to_csv('friends_mcq_w_answers_evaluation.csv',index=None)

,Question,Options,Correct Answer
0,How many times was Ross legally divorced? \n,"[Twice, Three times, Five times, Six times]",Three times
1,Where did Carol first meet Susan? \n,"[In college, At work, At the gym, At Central P...",At the gym
2,How did Susan and Ross come up with Ben’s name?,"[It was the doctor's name \n, They both had u...",It was on the janitor's name tag
3,What were Ben's first words? \n,"[Hi, Bye, Mom, Dumb]",Hi
4,How long did Ross and Emily date before they g...,"[14 days, 6 weeks \n, A year \n, 3 months]",6 weeks \n
...,...,...,...
692,Who is Monica?,"[Ross's Sister, Rachel's Sister, Emily's Siste...",Ross's Sister
693,"Who says it usually, ""I Know!""","[Rachel, Chandler, Monica, Joey]",Monica
694,Monica Kisses Joey,"[True, False]",False
695,Who did Phoebe marry?,"[David, Ross, Mike, Joey]",Mike


Pandas Apply:   0%|          | 0/695 [00:00<?, ?it/s]

In [97]:
data['number_of_options'].value_counts()

4    694
1      1
Name: number_of_options, dtype: int64